# Notebook to plot GAN quality statistics
* Pipeline to train models and create synthetic dataset:
    * create data with create_event_data_from_gazebase.py
    * train model with train_event_model.py
    * create_synthetic_data.py

In [1]:
import os
import numpy as np
import random
from tqdm import tqdm
import sys
import joblib
import seaborn as sns
import pandas as pd
from typing import Dict
from typing import List
from typing import Tuple



import tensorflow
import tensorflow as tf
from sp_eyegan.model import eventGAN as eventGAN
from sp_eyegan.model import vae_baseline as vae

%pylab inline
%load_ext autoreload
%autoreload 2

2023-04-03 14:49:15.627680: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-03 14:49:17.584684: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/gcc-4.4.3/objdir/x86_64-unknown-linux-gnu/libstdc++-v3/src/.libs/:/usr/local/gcc-4.4.3/objdir/x86_64-unknown-linux-gnu/libstdc++-v3/src/.libs/::/mnt/projekte/pmlcluster/home/prasse/projects/object_detection/opencv_multicore/opencv-2.4.4/lib:/usr/local/lib:/home/prasse/work/Projekte/tmp/maxi/bin/gsl/lib:/home/prasse/anaconda3/envs/tf/lib/:/mnt/projekte/pmlcluster/home/prasse/projects/obje

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


/home/prasse/anaconda3/envs/aeye_tensorflow_minimal/lib/python3.10/site-packages/IPython/core/magics/pylab.py:162: UserWarning: pylab import has clobbered these variables: ['random']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [2]:
def vel_to_dva(vel_data, x_start = 0,
             y_start = 0):
    x_vel = vel_data[:,0]
    y_vel = vel_data[:,1]
    x_px  = []
    y_px  = []
    cur_x_pos = x_start
    cur_y_pos = y_start
    for i in range(len(x_vel)):
        x_px.append(cur_x_pos + x_vel[i])
        y_px.append(cur_y_pos + y_vel[i])
        cur_x_pos = x_px[-1]
        cur_y_pos = y_px[-1]
    return np.concatenate([np.expand_dims(np.array(x_px),axis=1),
                           np.expand_dims(np.array(y_px),axis=1)],axis=1)

def get_fixation_stats(input_data):
    real_x_vels      = []
    real_y_vels      = []
    real_vels        = []
    real_dispersions = []
    cur_vel_data     = input_data
    for i in tqdm(np.arange(cur_vel_data.shape[0])):
        cur_vels = cur_vel_data[i]
        cur_dva = vel_to_dva(cur_vels)
        try:
            end_id = np.where(np.logical_and(cur_vels[:,0] == 0,
                                             cur_vels[:,1] ==0 ))[0][0]
        except:
            end_id = len(cur_vels)
        if end_id == 0:
            continue
        cur_vels = cur_vels[0:end_id]
        cur_dva = cur_dva[0:end_id]
        x_dva = cur_dva[:,0]
        y_dva = cur_dva[:,1]
        real_x_vels.append(cur_vels[:,0])
        real_y_vels.append(cur_vels[:,1])
        
        vels =  np.power(np.array(real_x_vels[-1]),2) +\
                np.power(np.array(real_y_vels[-1]),2)
        vels = np.sqrt(vels)
        real_vels.append(vels)
        x_amp = np.abs(np.max(x_dva) - np.min(x_dva))
        y_amp = np.abs(np.max(y_dva) - np.min(y_dva))
        cur_dispersion = x_amp + y_amp
        real_dispersions.append(cur_dispersion)
    
    return real_vels, real_x_vels,real_y_vels, real_dispersions


def get_saccade_stats(input_data, max_velocity = 0.5):
    real_x_vels      = []
    real_y_vels      = []
    real_vels        = []
    real_amplitudes  = []
    real_x_accs      = []
    real_y_accs      = []
    real_accs        = []
    cur_vel_data     = input_data
    for i in tqdm(np.arange(cur_vel_data.shape[0])):
        cur_vels = cur_vel_data[i]
        cur_dva = vel_to_dva(cur_vels)
        try:
            end_id = np.where(np.logical_and(cur_vels[:,0] == 0,
                                             cur_vels[:,1] ==0 ))[0][0]
        except:
            end_id = len(cur_vels)
        if end_id == 0:
            continue
        cur_vels = cur_vels[0:end_id]
        cur_dva = cur_dva[0:end_id]
        x_dva = cur_dva[:,0]
        y_dva = cur_dva[:,1]
        real_x_vels.append(cur_vels[:,0])
        real_y_vels.append(cur_vels[:,1])
        
        vels =  np.power(np.array(real_x_vels[-1]),2) +\
                np.power(np.array(real_y_vels[-1]),2)
        vels = np.sqrt(vels)
        real_vels.append(vels)        
        
        x_accs = cur_vels[1:,0] - cur_vels[:-1,0]
        y_accs = cur_vels[1:,1] - cur_vels[:-1,0]
        
        accs = np.power(np.array(x_accs),2) +\
                np.power(np.array(y_accs),2)
        accs = np.sqrt(accs)
        
        real_x_accs.append(x_accs)
        real_y_accs.append(y_accs)
        real_accs.append(accs)
        cur_complete_vels = np.sqrt(np.power(cur_vels[:,0],2) + np.power(cur_vels[:,1],2))
        cur_complete_vels[cur_complete_vels > max_velocity] = max_velocity
        cur_amplitude = np.sqrt(np.power(x_dva[0] - x_dva[-1],2) + np.power(y_dva[0] - y_dva[-1],2))
        real_amplitudes.append(cur_amplitude)
    
    return real_vels, real_x_vels, real_y_vels, real_accs, real_x_accs, real_y_accs, np.array(real_amplitudes)


# calculate the kl divergence
def kl_divergence(p, q):
    from math import log2
    from math import sqrt
    rel_etropies = [p[i] * log2(p[i]/q[i]) for i in range(len(p))]
    return np.sum(np.array(rel_etropies,dtype=np.float32))
 
# calculate the js divergence
def js_divergence(p, q):
    m = 0.5 * (p + q)
    return 0.5 * kl_divergence(p, m) + 0.5 * kl_divergence(q, m)


def draw_display(dispsize: Tuple[int, int], imagefile=None) -> Tuple[pyplot.figure, pyplot.Axes]:
    # construct screen (black background)
    # dots per inch
    img = image.imread(imagefile)
    dpi = 100.0
    # determine the figure size in inches
    figsize = (dispsize[0]/dpi, dispsize[1]/dpi)
    # create a figure
    fig = pyplot.figure(figsize=figsize, dpi=dpi, frameon=False)
    ax = pyplot.Axes(fig, [0, 0, 1, 1])
    ax.set_axis_off()
    fig.add_axes(ax)
    # plot display
    ax.axis([0, dispsize[0], 0, dispsize[1]])
    ax.imshow(img)

    return fig, ax



def pix2deg(pix, screenPX,screenCM,distanceCM, adjust_origin=True):
    # Converts pixel screen coordinate to degrees of visual angle
    # screenPX is the number of pixels that the monitor has in the horizontal
    # axis (for x coord) or vertical axis (for y coord)
    # screenCM is the width of the monitor in centimeters
    # distanceCM is the distance of the monitor to the retina 
    # pix: screen coordinate in pixels
    # adjust origin: if origin (0,0) of screen coordinates is in the corner of the screen rather than in the center, set to True to center coordinates
    pix=np.array(pix)
    # center screen coordinates such that 0 is center of the screen:
    if adjust_origin: 
        pix = pix-(screenPX)/2 # pixel coordinates start with (0,0) 
    # eye-to-screen-distance in pixels of screen
    distancePX = distanceCM*(screenPX/screenCM)
    return np.arctan2(pix,distancePX) * 180/np.pi #  *180/pi wandelt bogenmass in grad


def deg2pix(deg, screenPX, screenCM, distanceCM, adjust_origin = True, offsetCM = 0):
    # Converts degrees of visual angle to pixel screen coordinates
    # screenPX is the number of pixels that the monitor has in the horizontal
    # screenCM is the width of the monitor in centimeters
    # distanceCM is the distance of the monitor to the retina 
    phi = np.arctan2(1,distanceCM)*180/np.pi
    pix = deg/(phi/(screenPX/(screenCM)))
    if adjust_origin:
        pix += (screenPX/2)
    if offsetCM != 0:
        offsetPX = offsetCM*(screenPX/screenCM)
        pix += offsetPX
    return pix

In [3]:
# params
GPU = 0
data_dir = 'data/'
plot_dir = 'plots/'
result_dir = 'results/'

model_name = 'GAN'
real_name = 'real'
gauss_name = 'Gauss'


# load data
column_dict = joblib.load(data_dir + 'column_dict.joblib')
fixation_matrix = np.load(data_dir + 'fixation_matrix_gazebase_vd_text.npy')
saccade_matrix = np.load(data_dir  + 'saccade_matrix_gazebase_vd_text.npy')

fix_window_size = fixation_matrix.shape[1]
sac_window_size = saccade_matrix.shape[1]
gen_kernel_sizes_fixation = [fix_window_size,8,4,2]
gen_kernel_sizes_saccade = [sac_window_size,8,4,2]
    
# params for NN
random_size = 32
gen_filter_sizes = [16,8,4,2]
channels = 2
relu_in_last = False
batch_size = 256

dis_kernel_sizes = [8,16,32]
dis_fiter_sizes = [32,64,128]
dis_dropout = 0.3

sample_size = 1000
max_velocity = 0.5
seed = 42
np.random.seed(seed)

model_config_fixation = {'gen_kernel_sizes':gen_kernel_sizes_fixation,
                'gen_filter_sizes':gen_filter_sizes,
                'dis_kernel_sizes':dis_kernel_sizes,
                'dis_fiter_sizes':dis_fiter_sizes,
                'dis_dropout':dis_dropout,
                'window_size':fix_window_size,
                'channels':channels,
                'batch_size':batch_size,
                'random_size':random_size,
                'relu_in_last':relu_in_last,
               }

model_config_saccade = {'gen_kernel_sizes':gen_kernel_sizes_saccade,
                'gen_filter_sizes':gen_filter_sizes,
                'dis_kernel_sizes':dis_kernel_sizes,
                'dis_fiter_sizes':dis_fiter_sizes,
                'dis_dropout':dis_dropout,
                'window_size':sac_window_size,
                'channels':channels,
                'batch_size':batch_size,
                'random_size':random_size,
                'relu_in_last':relu_in_last,
               }


fixation_path  = 'event_model/fixation_model_text'
saccade_path   = 'event_model/saccade_model_text'

In [4]:
flag_train_on_gpu = True
if flag_train_on_gpu:
    import tensorflow as tf
    # select graphic card
    os.environ["CUDA_VISIBLE_DEVICES"] = str(GPU)
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    config = tf.compat.v1.ConfigProto(log_device_placement=True)
    config.gpu_options.per_process_gpu_memory_fraction = 1.
    config.gpu_options.allow_growth = True
    tf_session = tf.compat.v1.Session(config=config)
else:
    import tensorflow as tf
    # select graphic card
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

2023-04-03 14:49:20.996683: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:02:00.0, compute capability: 6.1



2023-04-03 14:49:23.244517: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 11178 MB memory:  -> device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:02:00.0, compute capability: 6.1


In [5]:
print(fixation_matrix.shape)
print(saccade_matrix.shape)

(393706, 100, 6)
(103890, 30, 6)


#### Data for statitical baseline

In [6]:
### select model for baseline method (statistical method)
gen_data_statitical = pd.read_csv('data/statistical_baseline_fix_data.txt',sep=';',header = None)
gen_data_statitical.columns = ['vel','type','frame']
gen_data_statitical.head()

,vel,type,frame
0,1.494492,2.0,2.0
1,470.000000,3.0,4.0
2,1.875850,2.0,6.0
3,2.185847,2.0,8.0
4,2.075208,2.0,10.0


In [7]:
fixation_index = 0
saccade_index = 1
velocities = np.array(gen_data_statitical['vel'])
event_type = np.array(gen_data_statitical['type'])
print('number fix points: ' + str(np.sum(event_type == fixation_index)))
print('number sac points: ' + str(np.sum(event_type == saccade_index)))

number fix points: 17889
number sac points: 5294


In [8]:
fix_vels_statistical = []
sac_vels_statistical = []
sac_acc_statistical  = []

# transfrom °/s to °/ms
scaling_factor = 1000.
prev_event = -1
cur_fixation = []
cur_saccade  = []
for i in tqdm(np.arange(len(velocities))):
    cur_event = event_type[i]
    cur_vel   = velocities[i]
    if cur_event == prev_event:
        if cur_event == fixation_index:
            cur_fixation.append(cur_vel/scaling_factor)
        elif cur_event == saccade_index:
            cur_saccade.append(cur_vel/scaling_factor)
    else:
        if prev_event == fixation_index and len(cur_fixation) > 0:
            fix_vels_statistical.append(np.array(cur_fixation))
        elif prev_event == saccade_index and len(cur_saccade) > 0:
            sac_vels_statistical.append(np.array(cur_saccade))
            if len(cur_saccade) > 2:
                sac_acc_statistical.append(np.array(cur_saccade)[1:] - np.array(cur_saccade)[0:-1])
        
        if cur_event == fixation_index:
            cur_fixation = [cur_vel/scaling_factor]
            cur_saccade = []
        elif cur_event == saccade_index:
            cur_saccade = [cur_vel/scaling_factor]
            cur_fixation = []
    prev_event = cur_event

100%|█████████████████████████████████| 56504/56504 [00:00<00:00, 703441.16it/s]


## Evaluate Fixations
* velocities, dispersion

In [9]:
from scipy.io import loadmat
eye_syn_fixation_data = loadmat('data/eyeSyn_velocities.mat')
eye_syn_fixation_data = eye_syn_fixation_data['out_data']

#### sample fixations with GAN

In [10]:
tf.keras.backend.clear_session()
fix_model = eventGAN.eventGAN(model_config_fixation)
fix_model.load_model(fixation_path)

2023-04-03 14:49:23.668190: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 11178 MB memory:  -> device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:02:00.0, compute capability: 6.1


In [11]:
noise = tf.random.normal([sample_size, random_size], seed = seed)
gen_fixations = np.array(fix_model.generator(noise, training=False),dtype=np.float32)

2023-04-03 14:49:24.425735: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2023-04-03 14:49:24.713681: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-04-03 14:49:24.716475: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 11.0.194, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.


In [12]:
print(gen_fixations.shape)
pd.DataFrame(gen_fixations.flatten()).describe()

(1000, 100, 2)


,0
count,200000.000000
mean,0.000072
std,0.004561
min,-0.046509
25%,-0.002635
50%,0.000125
75%,0.002811
max,0.029683


In [13]:
tf.keras.backend.clear_session()
vae_model = vae.VAE(vae.get_vae_encoder(64, 2, 2), vae.get_vae_decoder(2, 2))
vae_model.load_model('event_model/vae_fixation_10')
noise = tf.random.normal([sample_size * 2, 2], seed = seed)
vae_fixations = np.array(vae_model.decoder(noise, training=False),dtype=np.float32)
vae_fixations = np.concatenate([vae_fixations[0:sample_size],
                                vae_fixations[sample_size:]],axis=1)
vae_fixations = vae_fixations[:,0:100,:] -0.5

In [14]:
vae_fixations.shape
pd.DataFrame(vae_fixations.flatten()).describe()

,0
count,200000.000000
mean,0.000527
std,0.001529
min,-0.012177
25%,-0.000228
50%,0.000267
75%,0.001013
max,0.023295


In [15]:
rand_ids        = np.random.permutation(np.arange(fixation_matrix.shape[0]))
rand_ids_1      = rand_ids[0:sample_size]
rand_ids_2      = rand_ids[sample_size:sample_size+sample_size]
orig_fixations  = fixation_matrix[rand_ids_1,:,4:6]
orig_fixations[orig_fixations > max_velocity] = max_velocity
orig_fixations[orig_fixations < -max_velocity] = -max_velocity

orig_fixations_2 = fixation_matrix[rand_ids_2,:,4:6]
orig_fixations_2[orig_fixations_2 > max_velocity] = max_velocity
orig_fixations_2[orig_fixations_2 < -max_velocity] = -max_velocity

In [16]:
real_vels, real_x_vels, real_y_vels, real_dispersions = get_fixation_stats(orig_fixations)
real_vels_2, real_x_vels_2, real_y_vels_2, real_dispersions_2 = get_fixation_stats(orig_fixations_2)
fake_vels, fake_x_vels, fake_y_vels, fake_dispersions = get_fixation_stats(gen_fixations)
eyesyn_vels, eyesyn_x_vels, eyesyn_y_vels, eyesyn_dispersions =  get_fixation_stats(eye_syn_fixation_data)
vae_vels, vae_x_vels, vae_y_vels, vae_dispersions = get_fixation_stats(vae_fixations)

100%|█████████████████████████████████████| 1000/1000 [00:00<00:00, 6674.10it/s]


In [17]:
x_vel_mean = np.nanmean([item for sublist in real_x_vels for item in sublist])
x_vel_std = np.nanstd([item for sublist in real_x_vels for item in sublist])

y_vel_mean = np.nanmean([item for sublist in real_y_vels for item in sublist])
y_vel_std = np.nanstd([item for sublist in real_y_vels for item in sublist])
gauss_fixations = np.concatenate([np.random.normal(x_vel_mean,x_vel_std,(sample_size,fix_window_size,1)),
                                np.random.normal(y_vel_mean,y_vel_std,(sample_size,fix_window_size,1))],axis=2)
gauss_vels, gauss_x_vels, gauss_y_vels, gauss_dispersions = get_fixation_stats(gauss_fixations)

100%|█████████████████████████████████████| 1000/1000 [00:00<00:00, 8043.05it/s]


In [18]:
x_vel_mean = np.nanmean([item for sublist in real_x_vels for item in sublist])
x_vel_std = np.nanstd([item for sublist in real_x_vels for item in sublist])
print(x_vel_mean, x_vel_std)

y_vel_mean = np.nanmean([item for sublist in real_y_vels for item in sublist])
y_vel_std = np.nanstd([item for sublist in real_y_vels for item in sublist])
print(y_vel_mean, y_vel_std)

0.000588988012629895 0.012758497782424435
-6.230466548694232e-05 0.006571046669425702


### velocity
#### not normlized

In [19]:
real   = np.array([item for sublist in real_vels for item in sublist])
real_2 = np.array([item for sublist in real_vels_2 for item in sublist])
fake   = np.array([item for sublist in fake_vels for item in sublist])
gauss  = np.array([item for sublist in gauss_vels for item in sublist])
eyesyn = np.array([item for sublist in eyesyn_vels for item in sublist])
stat   = np.array([item for sublist in fix_vels_statistical for item in sublist])
f_vae  = np.array([item for sublist in vae_vels for item in sublist])

min_val = np.min(list(real) + list(fake) + list(gauss) + list(real_2) + list(eyesyn) + list(stat) + list(f_vae))
max_val = np.max(list(real) + list(fake) + list(gauss) + list(real_2) + list(eyesyn) + list(stat) + list(f_vae))
num_bins = 100
epsilon = 0.00001
bins = np.linspace(min_val,max_val,num_bins)

real_counts,_   = np.histogram(real, bins = bins, density=True)
real_counts_2,_   = np.histogram(real_2, bins = bins, density=True)
fake_counts,_   = np.histogram(fake, bins = bins, density=True)
gauss_counts,_  = np.histogram(gauss, bins = bins, density=True)
eyesyn_counts,_  = np.histogram(eyesyn, bins = bins, density=True)
stat_counts,_  = np.histogram(stat, bins = bins, density=True)
vae_counts,_  = np.histogram(f_vae, bins = bins, density=True)


js_divergence_vel_fake  = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(fake_counts) + epsilon)

js_divergence_vel_gauss = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(gauss_counts) + epsilon)

js_divergence_vel_real = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(real_counts_2) + epsilon)

js_divergence_vel_syneye = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(eyesyn_counts) + epsilon)

js_divergence_vel_stat = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(stat_counts) + epsilon)

js_divergence_vel_vae = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(vae_counts) + epsilon)


print('JS(real || GAN): ' + str(np.round(js_divergence_vel_fake,decimals = 2)))
print('JS(real || Gauss): ' + str(np.round(js_divergence_vel_gauss,decimals=2)))
print('JS(real || real): ' + str(np.round(js_divergence_vel_real,decimals = 2)))
print('JS(real || EyeSyn): ' + str(np.round(js_divergence_vel_syneye,decimals = 2)))
print('JS(real || stat): ' + str(np.round(js_divergence_vel_stat,decimals = 2)))
print('JS(real || VAE): ' + str(np.round(js_divergence_vel_vae,decimals = 2)))

JS(real || GAN): 5.56
JS(real || Gauss): 34.43
JS(real || real): 0.03
JS(real || EyeSyn): 12.27
JS(real || stat): 53.49
JS(real || VAE): 38.06


##### normlized

In [20]:
real   = np.array([item for sublist in real_vels for item in sublist])
real_2 = np.array([item for sublist in real_vels_2 for item in sublist])
fake   = np.array([item for sublist in fake_vels for item in sublist])
gauss  = np.array([item for sublist in gauss_vels for item in sublist])
eyesyn = np.array([item for sublist in eyesyn_vels for item in sublist])
stat   = np.array([item for sublist in fix_vels_statistical for item in sublist])
f_vae  = np.array([item for sublist in vae_vels for item in sublist])

min_val = np.min(list(real) + list(fake) + list(gauss) + list(real_2) + list(eyesyn) + list(stat) + list(f_vae))
max_val = np.max(list(real) + list(fake) + list(gauss) + list(real_2) + list(eyesyn) + list(stat) + list(f_vae))
num_bins = 100
epsilon = 0.00001
bins = np.linspace(min_val,max_val,num_bins)

real_counts,_   = np.histogram(real, bins = bins, density=True)
real_counts /= np.sum(real_counts)
real_counts_2,_   = np.histogram(real_2, bins = bins, density=True)
real_counts_2 /= np.sum(real_counts_2)
fake_counts,_   = np.histogram(fake, bins = bins, density=True)
fake_counts /= np.sum(fake_counts)
gauss_counts,_  = np.histogram(gauss, bins = bins, density=True)
gauss_counts /= np.sum(gauss_counts)
eyesyn_counts,_  = np.histogram(eyesyn, bins = bins, density=True)
eyesyn_counts /= np.sum(eyesyn_counts)
stat_counts,_  = np.histogram(stat, bins = bins, density=True)
stat_counts /= np.sum(stat_counts)
vae_counts,_  = np.histogram(f_vae, bins = bins, density=True)
vae_counts /= np.sum(vae_counts)


js_divergence_vel_fake  = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(fake_counts) + epsilon)

js_divergence_vel_gauss = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(gauss_counts) + epsilon)

js_divergence_vel_real = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(real_counts_2) + epsilon)

js_divergence_vel_syneye = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(eyesyn_counts) + epsilon)

js_divergence_vel_stat = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(stat_counts) + epsilon)

js_divergence_vel_vae = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(vae_counts) + epsilon)


print('JS(real || GAN): ' + str(np.round(js_divergence_vel_fake,decimals = 3)))
print('JS(real || Gauss): ' + str(np.round(js_divergence_vel_gauss,decimals=3)))
print('JS(real || real): ' + str(np.round(js_divergence_vel_real,decimals = 3)))
print('JS(real || EyeSyn): ' + str(np.round(js_divergence_vel_syneye,decimals = 3)))
print('JS(real || stat): ' + str(np.round(js_divergence_vel_stat,decimals = 3)))
print('JS(real || VAE): ' + str(np.round(js_divergence_vel_vae,decimals = 3)))

JS(real || GAN): 0.029
JS(real || Gauss): 0.182
JS(real || real): 0.0
JS(real || EyeSyn): 0.064
JS(real || stat): 0.283
JS(real || VAE): 0.201


### mean velocity
#### not normalized

In [21]:
real  = np.array([np.mean(a) for a in real_vels])
real_2 = np.array([np.mean(a) for a in real_vels_2])
fake  = np.array([np.mean(a) for a in fake_vels])
gauss = np.array([np.mean(a) for a in gauss_vels])
eyesyn = np.array([np.mean(a) for a in eyesyn_vels])
f_vae  = np.array([np.mean(a) for a in vae_vels])

min_val = np.min(list(real) + list(fake) + list(gauss) + list(real_2) + list(eyesyn) + list(stat) + list(f_vae))
max_val = np.max(list(real) + list(fake) + list(gauss) + list(real_2) + list(eyesyn) + list(stat) + list(f_vae))
num_bins = 100
epsilon = 0.00001
bins = np.linspace(min_val,max_val,num_bins)

real_counts,_   = np.histogram(real, bins = bins, density=True)
real_counts_2,_   = np.histogram(real_2, bins = bins, density=True)
fake_counts,_   = np.histogram(fake, bins = bins, density=True)
gauss_counts,_  = np.histogram(gauss, bins = bins, density=True)
eyesyn_counts,_  = np.histogram(eyesyn, bins = bins, density=True)
stat_counts,_  = np.histogram(stat, bins = bins, density=True)
vae_counts,_  = np.histogram(f_vae, bins = bins, density=True)


js_divergence_vel_fake  = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(fake_counts) + epsilon)

js_divergence_vel_gauss = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(gauss_counts) + epsilon)

js_divergence_vel_real = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(real_counts_2) + epsilon)

js_divergence_vel_syneye = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(eyesyn_counts) + epsilon)

js_divergence_vel_stat = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(stat_counts) + epsilon)

js_divergence_vel_vae = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(vae_counts) + epsilon)


print('JS(real || GAN): ' + str(np.round(js_divergence_vel_fake,decimals = 2)))
print('JS(real || Gauss): ' + str(np.round(js_divergence_vel_gauss,decimals=2)))
print('JS(real || real): ' + str(np.round(js_divergence_vel_real,decimals = 2)))
print('JS(real || EyeSyn): ' + str(np.round(js_divergence_vel_syneye,decimals = 2)))
print('JS(real || stat): ' + str(np.round(js_divergence_vel_stat,decimals = 2)))
print('JS(real || VAE): ' + str(np.round(js_divergence_vel_vae,decimals = 2)))

JS(real || GAN): 1208.91
JS(real || Gauss): 2683.12
JS(real || real): 109.9
JS(real || EyeSyn): 3213.22
JS(real || stat): 2779.17
JS(real || VAE): 3869.9


#### normalized

In [22]:
real  = np.array([np.mean(a) for a in real_vels])
real_2 = np.array([np.mean(a) for a in real_vels_2])
fake  = np.array([np.mean(a) for a in fake_vels])
gauss = np.array([np.mean(a) for a in gauss_vels])
eyesyn = np.array([np.mean(a) for a in eyesyn_vels])
f_vae  = np.array([np.mean(a) for a in vae_vels])

min_val = np.min(list(real) + list(fake) + list(gauss) + list(real_2) + list(eyesyn) + list(stat) + list(f_vae))
max_val = np.max(list(real) + list(fake) + list(gauss) + list(real_2) + list(eyesyn) + list(stat) + list(f_vae))
num_bins = 100
epsilon = 0.00001
bins = np.linspace(min_val,max_val,num_bins)

real_counts,_   = np.histogram(real, bins = bins, density=True)
real_counts /= np.sum(real_counts)
real_counts_2,_   = np.histogram(real_2, bins = bins, density=True)
real_counts_2 /= np.sum(real_counts_2)
fake_counts,_   = np.histogram(fake, bins = bins, density=True)
fake_counts /= np.sum(fake_counts)
gauss_counts,_  = np.histogram(gauss, bins = bins, density=True)
gauss_counts /= np.sum(gauss_counts)
eyesyn_counts,_  = np.histogram(eyesyn, bins = bins, density=True)
eyesyn_counts /= np.sum(eyesyn_counts)
stat_counts,_  = np.histogram(stat, bins = bins, density=True)
stat_counts /= np.sum(stat_counts)
vae_counts,_  = np.histogram(f_vae, bins = bins, density=True)
vae_counts /= np.sum(vae_counts)


js_divergence_vel_fake  = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(fake_counts) + epsilon)

js_divergence_vel_gauss = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(gauss_counts) + epsilon)

js_divergence_vel_real = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(real_counts_2) + epsilon)

js_divergence_vel_syneye = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(eyesyn_counts) + epsilon)

js_divergence_vel_stat = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(stat_counts) + epsilon)

js_divergence_vel_vae = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(vae_counts) + epsilon)


print('JS(real || GAN): ' + str(np.round(js_divergence_vel_fake,decimals = 3)))
print('JS(real || Gauss): ' + str(np.round(js_divergence_vel_gauss,decimals=3)))
print('JS(real || real): ' + str(np.round(js_divergence_vel_real,decimals = 3)))
print('JS(real || EyeSyn): ' + str(np.round(js_divergence_vel_syneye,decimals = 3)))
print('JS(real || stat): ' + str(np.round(js_divergence_vel_stat,decimals = 3)))
print('JS(real || VAE): ' + str(np.round(js_divergence_vel_vae,decimals = 3)))

JS(real || GAN): 0.295
JS(real || Gauss): 0.656
JS(real || real): 0.026
JS(real || EyeSyn): 0.785
JS(real || stat): 0.679
JS(real || VAE): 0.946


### dispersion
#### not normalized

In [23]:
real   = np.array(real_dispersions)
real_2 = np.array(real_dispersions_2)
fake   = np.array(fake_dispersions)
gauss  = np.array(gauss_dispersions)
eyesyn = np.array(eyesyn_dispersions)
f_vae  = np.array(vae_dispersions)

min_val = np.min(list(real) + list(fake) + list(gauss) + list(real_2) + list(eyesyn) + list(f_vae))
max_val = np.max(list(real) + list(fake) + list(gauss) + list(real_2) + list(eyesyn) + list(f_vae))
num_bins = 100
epsilon = 0.00001
bins = np.linspace(min_val,max_val,num_bins)

real_counts,_   = np.histogram(real, bins = bins, density=True)
real_counts_2,_   = np.histogram(real_2, bins = bins, density=True)
fake_counts,_   = np.histogram(fake, bins = bins, density=True)
gauss_counts,_  = np.histogram(gauss, bins = bins, density=True)
eyesyn_counts,_  = np.histogram(eyesyn, bins = bins, density=True)
vae_counts,_  = np.histogram(f_vae, bins = bins, density=True)



js_divergence_vel_fake  = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(fake_counts) + epsilon)

js_divergence_vel_gauss = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(gauss_counts) + epsilon)

js_divergence_vel_real = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(real_counts_2) + epsilon)

js_divergence_vel_syneye = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(eyesyn_counts) + epsilon)

js_divergence_vel_vae = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(vae_counts) + epsilon)

print('JS(real || GAN): ' + str(np.round(js_divergence_vel_fake,decimals = 2)))
print('JS(real || Gauss): ' + str(np.round(js_divergence_vel_gauss,decimals=2)))
print('JS(real || real): ' + str(np.round(js_divergence_vel_real,decimals = 2)))
print('JS(real || EyeSyn): ' + str(np.round(js_divergence_vel_syneye,decimals = 2)))
print('JS(real || VAE): ' + str(np.round(js_divergence_vel_vae,decimals = 2)))

JS(real || GAN): 45.77
JS(real || Gauss): 58.01
JS(real || real): 7.65
JS(real || EyeSyn): 166.86
JS(real || VAE): 121.82


#### normalized

In [24]:
real   = np.array(real_dispersions)
real_2 = np.array(real_dispersions_2)
fake   = np.array(fake_dispersions)
gauss  = np.array(gauss_dispersions)
eyesyn = np.array(eyesyn_dispersions)
f_vae  = np.array(vae_dispersions)

min_val = np.min(list(real) + list(fake) + list(gauss) + list(real_2) + list(eyesyn) + list(f_vae))
max_val = np.max(list(real) + list(fake) + list(gauss) + list(real_2) + list(eyesyn) + list(f_vae))
num_bins = 100
epsilon = 0.00001
bins = np.linspace(min_val,max_val,num_bins)

real_counts,_   = np.histogram(real, bins = bins, density=True)
real_counts /= np.sum(real_counts)
real_counts_2,_   = np.histogram(real_2, bins = bins, density=True)
real_counts_2 /= np.sum(real_counts_2)
fake_counts,_   = np.histogram(fake, bins = bins, density=True)
fake_counts /= np.sum(fake_counts)
gauss_counts,_  = np.histogram(gauss, bins = bins, density=True)
gauss_counts /= np.sum(gauss_counts)
eyesyn_counts,_  = np.histogram(eyesyn, bins = bins, density=True)
eyesyn_counts /= np.sum(eyesyn_counts)
stat_counts,_  = np.histogram(stat, bins = bins, density=True)
stat_counts /= np.sum(stat_counts)
vae_counts,_  = np.histogram(f_vae, bins = bins, density=True)
vae_counts /= np.sum(vae_counts)


js_divergence_vel_fake  = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(fake_counts) + epsilon)

js_divergence_vel_gauss = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(gauss_counts) + epsilon)

js_divergence_vel_real = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(real_counts_2) + epsilon)

js_divergence_vel_syneye = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(eyesyn_counts) + epsilon)

js_divergence_vel_stat = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(stat_counts) + epsilon)

js_divergence_vel_vae = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(vae_counts) + epsilon)


print('JS(real || GAN): ' + str(np.round(js_divergence_vel_fake,decimals = 3)))
print('JS(real || Gauss): ' + str(np.round(js_divergence_vel_gauss,decimals=3)))
print('JS(real || real): ' + str(np.round(js_divergence_vel_real,decimals = 3)))
print('JS(real || EyeSyn): ' + str(np.round(js_divergence_vel_syneye,decimals = 3)))
print('JS(real || VAE): ' + str(np.round(js_divergence_vel_vae,decimals = 3)))

JS(real || GAN): 0.271
JS(real || Gauss): 0.344
JS(real || real): 0.045
JS(real || EyeSyn): 0.989
JS(real || VAE): 0.722


/home/prasse/anaconda3/envs/aeye_tensorflow_minimal/lib/python3.10/site-packages/numpy/lib/histograms.py:906: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.sum(), bin_edges


## Evaluate Saccades
* peak, mean, median velocity
* peak, mean, median acceleration (peak mean)
* amplitude

In [25]:
tf.keras.backend.clear_session()
sac_model = eventGAN.eventGAN(model_config_saccade)
sac_model.load_model(saccade_path)

In [26]:
noise = tf.random.normal([sample_size, random_size], seed = seed)
gen_saccades = np.array(sac_model.generator(noise, training=False),dtype=np.float32)

In [27]:
print(gen_saccades.shape)
pd.DataFrame(gen_saccades.flatten()).describe()

(1000, 30, 2)


,0
count,60000.000000
mean,0.032931
std,0.082144
min,-0.311250
25%,-0.010363
50%,0.010359
75%,0.050939
max,0.837081


In [28]:
tf.keras.backend.clear_session()
vae_model = vae.VAE(vae.get_vae_encoder(64, 2, 2), vae.get_vae_decoder(2, 2))
vae_model.load_model('event_model/vae_saccade_10')
noise = tf.random.normal([sample_size, 2], seed = seed)
vae_saccades = np.array(vae_model.decoder(noise, training=False),dtype=np.float32)
vae_saccades = vae_saccades[:,0:30,:] -0.5

2023-04-03 14:49:28.361098: W tensorflow/tsl/framework/bfc_allocator.cc:360] Garbage collection: deallocate free memory regions (i.e., allocations) so that we can re-allocate a larger region to avoid OOM due to memory fragmentation. If you see this message frequently, you are running near the threshold of the available device memory and re-allocation may incur great performance overhead. You may try smaller batch sizes to observe the performance impact. Set TF_ENABLE_GPU_GARBAGE_COLLECTION=false if you'd like to disable this feature.


In [29]:
print(vae_saccades.shape)
pd.DataFrame(vae_saccades.flatten()).describe()

(1000, 30, 2)


,0
count,60000.000000
mean,-0.043367
std,0.048688
min,-0.118281
25%,-0.094478
50%,-0.037366
75%,0.005859
max,0.015778


In [30]:
rand_ids        = np.random.permutation(np.arange(saccade_matrix.shape[0]))
rand_ids_1      = rand_ids[0:sample_size]
rand_ids_2      = rand_ids[sample_size:sample_size+sample_size]

rand_ids        = np.arange(saccade_matrix.shape[0])
rand_ids        = np.random.permutation(rand_ids)[0:sample_size]
orig_saccades   = saccade_matrix[rand_ids_1,:,4:6]
orig_saccades[orig_saccades > max_velocity] = max_velocity
orig_saccades[orig_saccades < -max_velocity] = -max_velocity

orig_saccades_2 = saccade_matrix[rand_ids_2,:,4:6]
orig_saccades_2[orig_saccades_2 > max_velocity] = max_velocity
orig_saccades_2[orig_saccades_2 < -max_velocity] = -max_velocity

In [31]:
print(orig_saccades.shape)
print(orig_saccades_2.shape)
print(saccade_matrix.shape)

(1000, 30, 2)
(1000, 30, 2)
(103890, 30, 6)


In [32]:
real_vels_sac, real_x_vels_sac, real_y_vels_sac, real_accs_sac, real_x_accs_sac, real_y_accs_sac, real_amplitudes_sac      = get_saccade_stats(orig_saccades)
real_vels_sac_2, real_x_vels_sac_2, real_y_vels_sac_2, real_accs_sac_2, real_x_accs_sac_2, real_y_accs_sac_2, real_amplitudes_sac_2      = get_saccade_stats(orig_saccades_2)
fake_vels_sac, fake_x_vels_sac, fake_y_vels_sac, fake_accs_sac, fake_real_x_accs_sac, fake_y_accs_sac, fake_amplitudes_sac = get_saccade_stats(gen_saccades)
vae_vels_sac, vae_x_vels_sac, vae_y_vels_sac, vae_accs_sac, vae_x_accs_sac, vae_y_accs_sac, vae_amplitudes_sac = get_saccade_stats(vae_saccades)

100%|████████████████████████████████████| 1000/1000 [00:00<00:00, 11068.31it/s]


In [33]:
x_vel_mean_sac = np.nanmean([item for sublist in real_x_vels_sac for item in sublist])
x_vel_std_sac = np.nanstd([item for sublist in real_x_vels_sac for item in sublist])

y_vel_mean_sac = np.nanmean([item for sublist in real_y_vels_sac for item in sublist])
y_vel_std_sac = np.nanstd([item for sublist in real_y_vels_sac for item in sublist])
gauss_saccades = np.concatenate([np.random.normal(x_vel_mean_sac,x_vel_std_sac,(sample_size,sac_window_size,1)),
                                np.random.normal(y_vel_mean_sac,y_vel_std_sac,(sample_size,sac_window_size,1))],axis=2)
gauss_vels_sac, gauss_x_vels_sac, gauss_y_vels_sac, gauss_accs_sac, gauss_real_x_accs_sac, gauss_y_accs_sac, gauss_amplitudes_sac = get_saccade_stats(gauss_saccades)

100%|████████████████████████████████████| 1000/1000 [00:00<00:00, 12680.31it/s]


### peak velocity
#### not normalized

In [34]:
real = np.array([np.max(a) for a in real_vels_sac])
real_2 = np.array([np.max(a) for a in real_vels_sac_2])
fake = np.array([np.max(a) for a in fake_vels_sac])
gauss = np.array([np.max(a) for a in gauss_vels_sac])
stat   = np.array([np.max(a) for a in sac_vels_statistical])
f_vae  = np.array([np.max(a) for a in vae_vels_sac])

min_val = np.min(list(real) + list(fake) + list(gauss) + list(real_2) + list(eyesyn) + list(stat) + list(f_vae))
max_val = np.max(list(real) + list(fake) + list(gauss) + list(real_2) + list(eyesyn) + list(stat) + list(f_vae))
num_bins = 100
epsilon = 0.00001
bins = np.linspace(min_val,max_val,num_bins)

real_counts,_   = np.histogram(real, bins = bins, density=True)
real_counts_2,_   = np.histogram(real_2, bins = bins, density=True)
fake_counts,_   = np.histogram(fake, bins = bins, density=True)
gauss_counts,_  = np.histogram(gauss, bins = bins, density=True)
eyesyn_counts,_  = np.histogram(eyesyn, bins = bins, density=True)
stat_counts,_  = np.histogram(stat, bins = bins, density=True)
vae_counts,_  = np.histogram(f_vae, bins = bins, density=True)


js_divergence_vel_fake  = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(fake_counts) + epsilon)

js_divergence_vel_gauss = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(gauss_counts) + epsilon)

js_divergence_vel_real = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(real_counts_2) + epsilon)

js_divergence_vel_syneye = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(eyesyn_counts) + epsilon)

js_divergence_vel_stat = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(stat_counts) + epsilon)

js_divergence_vel_vae = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(vae_counts) + epsilon)


print('JS(real || GAN): ' + str(np.round(js_divergence_vel_fake,decimals = 2)))
print('JS(real || Gauss): ' + str(np.round(js_divergence_vel_gauss,decimals=2)))
print('JS(real || real): ' + str(np.round(js_divergence_vel_real,decimals = 2)))
print('JS(real || EyeSyn): ' + str(np.round(js_divergence_vel_syneye,decimals = 2)))
print('JS(real || stat): ' + str(np.round(js_divergence_vel_stat,decimals = 2)))
print('JS(real || VAE): ' + str(np.round(js_divergence_vel_vae,decimals = 2)))

JS(real || GAN): 37.29
JS(real || Gauss): 43.26
JS(real || real): 2.99
JS(real || EyeSyn): 101.25
JS(real || stat): 39.0
JS(real || VAE): 104.4


#### normalized

In [35]:
real = np.array([np.max(a) for a in real_vels_sac])
real_2 = np.array([np.max(a) for a in real_vels_sac_2])
fake = np.array([np.max(a) for a in fake_vels_sac])
gauss = np.array([np.max(a) for a in gauss_vels_sac])
stat   = np.array([np.max(a) for a in sac_vels_statistical])
f_vae  = np.array([np.max(a) for a in vae_vels_sac])

min_val = np.min(list(real) + list(fake) + list(gauss) + list(real_2) + list(eyesyn) + list(stat) + list(f_vae))
max_val = np.max(list(real) + list(fake) + list(gauss) + list(real_2) + list(eyesyn) + list(stat) + list(f_vae))
num_bins = 100
epsilon = 0.00001
bins = np.linspace(min_val,max_val,num_bins)

real_counts,_   = np.histogram(real, bins = bins, density=True)
real_counts /= np.sum(real_counts)
real_counts_2,_   = np.histogram(real_2, bins = bins, density=True)
real_counts_2 /= np.sum(real_counts_2)
fake_counts,_   = np.histogram(fake, bins = bins, density=True)
fake_counts /= np.sum(fake_counts)
gauss_counts,_  = np.histogram(gauss, bins = bins, density=True)
gauss_counts /= np.sum(gauss_counts)
eyesyn_counts,_  = np.histogram(eyesyn, bins = bins, density=True)
eyesyn_counts /= np.sum(eyesyn_counts)
stat_counts,_  = np.histogram(stat, bins = bins, density=True)
stat_counts /= np.sum(stat_counts)
vae_counts,_  = np.histogram(f_vae, bins = bins, density=True)
vae_counts /= np.sum(vae_counts)


js_divergence_vel_fake  = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(fake_counts) + epsilon)

js_divergence_vel_gauss = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(gauss_counts) + epsilon)

js_divergence_vel_real = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(real_counts_2) + epsilon)

js_divergence_vel_syneye = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(eyesyn_counts) + epsilon)

js_divergence_vel_stat = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(stat_counts) + epsilon)

js_divergence_vel_vae = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(vae_counts) + epsilon)


print('JS(real || GAN): ' + str(np.round(js_divergence_vel_fake,decimals = 3)))
print('JS(real || Gauss): ' + str(np.round(js_divergence_vel_gauss,decimals=3)))
print('JS(real || real): ' + str(np.round(js_divergence_vel_real,decimals = 3)))
print('JS(real || EyeSyn): ' + str(np.round(js_divergence_vel_syneye,decimals = 3)))
print('JS(real || stat): ' + str(np.round(js_divergence_vel_stat,decimals = 3)))
print('JS(real || VAE): ' + str(np.round(js_divergence_vel_vae,decimals = 3)))

JS(real || GAN): 0.33
JS(real || Gauss): 0.382
JS(real || real): 0.026
JS(real || EyeSyn): 0.896
JS(real || stat): 0.346
JS(real || VAE): 0.924


### mean velocity
#### not normalized

In [36]:
real = np.array([np.mean(a) for a in real_vels_sac])
real_2 = np.array([np.mean(a) for a in real_vels_sac_2])
fake = np.array([np.mean(a) for a in fake_vels_sac])
gauss = np.array([np.mean(a) for a in gauss_vels_sac])
stat   = np.array([np.mean(a) for a in sac_vels_statistical])
f_vae = np.array([np.mean(a) for a in vae_vels_sac])

min_val = np.min(list(real) + list(fake) + list(gauss) + list(real_2) + list(eyesyn) + list(stat) + list(f_vae))
max_val = np.max(list(real) + list(fake) + list(gauss) + list(real_2) + list(eyesyn) + list(stat) + list(f_vae))
num_bins = 100
epsilon = 0.00001
bins = np.linspace(min_val,max_val,num_bins)

real_counts,_   = np.histogram(real, bins = bins, density=True)
real_counts_2,_   = np.histogram(real_2, bins = bins, density=True)
fake_counts,_   = np.histogram(fake, bins = bins, density=True)
gauss_counts,_  = np.histogram(gauss, bins = bins, density=True)
eyesyn_counts,_  = np.histogram(eyesyn, bins = bins, density=True)
stat_counts,_  = np.histogram(stat, bins = bins, density=True)
vae_counts,_  = np.histogram(f_vae, bins = bins, density=True)


js_divergence_vel_fake  = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(fake_counts) + epsilon)

js_divergence_vel_gauss = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(gauss_counts) + epsilon)

js_divergence_vel_real = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(real_counts_2) + epsilon)

js_divergence_vel_syneye = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(eyesyn_counts) + epsilon)

js_divergence_vel_stat = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(stat_counts) + epsilon)

js_divergence_vel_vae = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(vae_counts) + epsilon)


print('JS(real || GAN): ' + str(np.round(js_divergence_vel_fake,decimals = 2)))
print('JS(real || Gauss): ' + str(np.round(js_divergence_vel_gauss,decimals=2)))
print('JS(real || real): ' + str(np.round(js_divergence_vel_real,decimals = 2)))
print('JS(real || EyeSyn): ' + str(np.round(js_divergence_vel_syneye,decimals = 2)))
print('JS(real || stat): ' + str(np.round(js_divergence_vel_stat,decimals = 2)))
print('JS(real || VAE): ' + str(np.round(js_divergence_vel_vae,decimals = 2)))

JS(real || GAN): 27.08
JS(real || Gauss): 66.7
JS(real || real): 2.52
JS(real || EyeSyn): 95.26
JS(real || stat): 26.75
JS(real || VAE): 109.48


#### normalized

In [37]:
real = np.array([np.mean(a) for a in real_vels_sac])
real_2 = np.array([np.mean(a) for a in real_vels_sac_2])
fake = np.array([np.mean(a) for a in fake_vels_sac])
gauss = np.array([np.mean(a) for a in gauss_vels_sac])
stat   = np.array([np.mean(a) for a in sac_vels_statistical])
f_vae = np.array([np.mean(a) for a in vae_vels_sac])

min_val = np.min(list(real) + list(fake) + list(gauss) + list(real_2) + list(eyesyn) + list(stat) + list(f_vae))
max_val = np.max(list(real) + list(fake) + list(gauss) + list(real_2) + list(eyesyn) + list(stat) + list(f_vae))
num_bins = 100
epsilon = 0.00001
bins = np.linspace(min_val,max_val,num_bins)

real_counts,_   = np.histogram(real, bins = bins, density=True)
real_counts /= np.sum(real_counts)
real_counts_2,_   = np.histogram(real_2, bins = bins, density=True)
real_counts_2 /= np.sum(real_counts_2)
fake_counts,_   = np.histogram(fake, bins = bins, density=True)
fake_counts /= np.sum(fake_counts)
gauss_counts,_  = np.histogram(gauss, bins = bins, density=True)
gauss_counts /= np.sum(gauss_counts)
eyesyn_counts,_  = np.histogram(eyesyn, bins = bins, density=True)
eyesyn_counts /= np.sum(eyesyn_counts)
stat_counts,_  = np.histogram(stat, bins = bins, density=True)
stat_counts /= np.sum(stat_counts)
vae_counts,_  = np.histogram(f_vae, bins = bins, density=True)
vae_counts /= np.sum(vae_counts)


js_divergence_vel_fake  = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(fake_counts) + epsilon)

js_divergence_vel_gauss = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(gauss_counts) + epsilon)

js_divergence_vel_real = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(real_counts_2) + epsilon)

js_divergence_vel_syneye = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(eyesyn_counts) + epsilon)

js_divergence_vel_stat = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(stat_counts) + epsilon)

js_divergence_vel_vae = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(vae_counts) + epsilon)


print('JS(real || GAN): ' + str(np.round(js_divergence_vel_fake,decimals = 3)))
print('JS(real || Gauss): ' + str(np.round(js_divergence_vel_gauss,decimals=3)))
print('JS(real || real): ' + str(np.round(js_divergence_vel_real,decimals = 3)))
print('JS(real || EyeSyn): ' + str(np.round(js_divergence_vel_syneye,decimals = 3)))
print('JS(real || stat): ' + str(np.round(js_divergence_vel_stat,decimals = 3)))
print('JS(real || VAE): ' + str(np.round(js_divergence_vel_vae,decimals = 3)))

JS(real || GAN): 0.23
JS(real || Gauss): 0.566
JS(real || real): 0.021
JS(real || EyeSyn): 0.808
JS(real || stat): 0.226
JS(real || VAE): 0.929


### peak acceleration
#### not normalized

In [38]:
real = np.array([np.max(a) for a in real_accs_sac])
real_2 = np.array([np.mean(a) for a in real_accs_sac])
fake = np.array([np.max(a) for a in fake_accs_sac])
gauss = np.array([np.max(a) for a in gauss_accs_sac])
stat   = np.array([np.max(a) for a in sac_acc_statistical])
f_vae = np.array([np.max(a) for a in vae_accs_sac])

min_val = np.min(list(real) + list(fake) + list(gauss) + list(real_2) + list(eyesyn) + list(stat) + list(f_vae))
max_val = np.max(list(real) + list(fake) + list(gauss) + list(real_2) + list(eyesyn) + list(stat) + list(f_vae))
num_bins = 100
epsilon = 0.00001
bins = np.linspace(min_val,max_val,num_bins)

real_counts,_   = np.histogram(real, bins = bins, density=True)
real_counts_2,_   = np.histogram(real_2, bins = bins, density=True)
fake_counts,_   = np.histogram(fake, bins = bins, density=True)
gauss_counts,_  = np.histogram(gauss, bins = bins, density=True)
eyesyn_counts,_  = np.histogram(eyesyn, bins = bins, density=True)
stat_counts,_  = np.histogram(stat, bins = bins, density=True)
vae_counts,_  = np.histogram(f_vae, bins = bins, density=True)


js_divergence_vel_fake  = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(fake_counts) + epsilon)

js_divergence_vel_gauss = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(gauss_counts) + epsilon)

js_divergence_vel_real = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(real_counts_2) + epsilon)

js_divergence_vel_syneye = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(eyesyn_counts) + epsilon)

js_divergence_vel_stat = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(stat_counts) + epsilon)

js_divergence_vel_vae = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(vae_counts) + epsilon)


print('JS(real || GAN): ' + str(np.round(js_divergence_vel_fake,decimals = 2)))
print('JS(real || Gauss): ' + str(np.round(js_divergence_vel_gauss,decimals=2)))
print('JS(real || real): ' + str(np.round(js_divergence_vel_real,decimals = 2)))
print('JS(real || EyeSyn): ' + str(np.round(js_divergence_vel_syneye,decimals = 2)))
print('JS(real || stat): ' + str(np.round(js_divergence_vel_stat,decimals = 2)))
print('JS(real || VAE): ' + str(np.round(js_divergence_vel_vae,decimals = 2)))

JS(real || GAN): 19.25
JS(real || Gauss): 55.91
JS(real || real): 17.21
JS(real || EyeSyn): 64.52
JS(real || stat): 67.35
JS(real || VAE): 66.78


#### normalized

In [39]:
real = np.array([np.max(a) for a in real_accs_sac])
real_2 = np.array([np.mean(a) for a in real_accs_sac])
fake = np.array([np.max(a) for a in fake_accs_sac])
gauss = np.array([np.max(a) for a in gauss_accs_sac])
stat   = np.array([np.max(a) for a in sac_acc_statistical])
f_vae = np.array([np.max(a) for a in vae_accs_sac])

min_val = np.min(list(real) + list(fake) + list(gauss) + list(real_2) + list(eyesyn) + list(stat) + list(f_vae))
max_val = np.max(list(real) + list(fake) + list(gauss) + list(real_2) + list(eyesyn) + list(stat) + list(f_vae))
num_bins = 100
epsilon = 0.00001
bins = np.linspace(min_val,max_val,num_bins)

real_counts,_   = np.histogram(real, bins = bins, density=True)
real_counts /= np.sum(real_counts)
real_counts_2,_   = np.histogram(real_2, bins = bins, density=True)
real_counts_2 /= np.sum(real_counts_2)
fake_counts,_   = np.histogram(fake, bins = bins, density=True)
fake_counts /= np.sum(fake_counts)
gauss_counts,_  = np.histogram(gauss, bins = bins, density=True)
gauss_counts /= np.sum(gauss_counts)
eyesyn_counts,_  = np.histogram(eyesyn, bins = bins, density=True)
eyesyn_counts /= np.sum(eyesyn_counts)
stat_counts,_  = np.histogram(stat, bins = bins, density=True)
stat_counts /= np.sum(stat_counts)
vae_counts,_  = np.histogram(f_vae, bins = bins, density=True)
vae_counts /= np.sum(vae_counts)


js_divergence_vel_fake  = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(fake_counts) + epsilon)

js_divergence_vel_gauss = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(gauss_counts) + epsilon)

js_divergence_vel_real = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(real_counts_2) + epsilon)

js_divergence_vel_syneye = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(eyesyn_counts) + epsilon)

js_divergence_vel_stat = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(stat_counts) + epsilon)

js_divergence_vel_vae = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(vae_counts) + epsilon)


print('JS(real || GAN): ' + str(np.round(js_divergence_vel_fake,decimals = 3)))
print('JS(real || Gauss): ' + str(np.round(js_divergence_vel_gauss,decimals=3)))
print('JS(real || real): ' + str(np.round(js_divergence_vel_real,decimals = 3)))
print('JS(real || EyeSyn): ' + str(np.round(js_divergence_vel_syneye,decimals = 3)))
print('JS(real || stat): ' + str(np.round(js_divergence_vel_stat,decimals = 3)))
print('JS(real || VAE): ' + str(np.round(js_divergence_vel_vae,decimals = 3)))

JS(real || GAN): 0.263
JS(real || Gauss): 0.763
JS(real || real): 0.235
JS(real || EyeSyn): 0.881
JS(real || stat): 0.921
JS(real || VAE): 0.912


### mean acceleration
#### not normalized

In [45]:
real = np.array([np.mean(a) for a in real_accs_sac])
real_2 = np.array([np.mean(a) for a in real_accs_sac_2])
fake = np.array([np.mean(a) for a in fake_accs_sac])
gauss = np.array([np.mean(a) for a in gauss_accs_sac])
stat   = np.array([np.mean(a) for a in sac_acc_statistical])
f_vae = np.array([np.mean(a) for a in vae_accs_sac])

min_val = np.min(list(real) + list(fake) + list(gauss) + list(real_2) + list(eyesyn) + list(stat) + list(f_vae))
max_val = np.max(list(real) + list(fake) + list(gauss) + list(real_2) + list(eyesyn) + list(stat) + list(f_vae))
num_bins = 100
epsilon = 0.00001
bins = np.linspace(min_val,max_val,num_bins)

real_counts,_   = np.histogram(real, bins = bins, density=True)
real_counts_2,_   = np.histogram(real_2, bins = bins, density=True)
fake_counts,_   = np.histogram(fake, bins = bins, density=True)
gauss_counts,_  = np.histogram(gauss, bins = bins, density=True)
eyesyn_counts,_  = np.histogram(eyesyn, bins = bins, density=True)
stat_counts,_  = np.histogram(stat, bins = bins, density=True)
vae_counts,_  = np.histogram(f_vae, bins = bins, density=True)


js_divergence_vel_fake  = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(fake_counts) + epsilon)

js_divergence_vel_gauss = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(gauss_counts) + epsilon)

js_divergence_vel_real = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(real_counts_2) + epsilon)

js_divergence_vel_syneye = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(eyesyn_counts) + epsilon)

js_divergence_vel_stat = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(stat_counts) + epsilon)

js_divergence_vel_vae = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(vae_counts) + epsilon)


print('JS(real || GAN): ' + str(np.round(js_divergence_vel_fake,decimals = 2)))
print('JS(real || Gauss): ' + str(np.round(js_divergence_vel_gauss,decimals=2)))
print('JS(real || real): ' + str(np.round(js_divergence_vel_real,decimals = 2)))
print('JS(real || EyeSyn): ' + str(np.round(js_divergence_vel_syneye,decimals = 2)))
print('JS(real || stat): ' + str(np.round(js_divergence_vel_stat,decimals = 2)))
print('JS(real || VAE): ' + str(np.round(js_divergence_vel_vae,decimals = 2)))

JS(real || GAN): 31.95
JS(real || Gauss): 60.9
JS(real || real): 4.55
JS(real || EyeSyn): 117.68
JS(real || stat): 124.19
JS(real || VAE): 131.52


#### normalized

In [47]:
real = np.array([np.mean(a) for a in real_accs_sac])
real_2 = np.array([np.mean(a) for a in real_accs_sac_2])
fake = np.array([np.mean(a) for a in fake_accs_sac])
gauss = np.array([np.mean(a) for a in gauss_accs_sac])
stat   = np.array([np.mean(a) for a in sac_acc_statistical])
f_vae = np.array([np.mean(a) for a in vae_accs_sac])

min_val = np.min(list(real) + list(fake) + list(gauss) + list(real_2) + list(eyesyn) + list(stat) + list(f_vae))
max_val = np.max(list(real) + list(fake) + list(gauss) + list(real_2) + list(eyesyn) + list(stat) + list(f_vae))
num_bins = 100
epsilon = 0.00001
bins = np.linspace(min_val,max_val,num_bins)

real_counts,_   = np.histogram(real, bins = bins, density=True)
real_counts /= np.sum(real_counts)
real_counts_2,_   = np.histogram(real_2, bins = bins, density=True)
real_counts_2 /= np.sum(real_counts_2)
fake_counts,_   = np.histogram(fake, bins = bins, density=True)
fake_counts /= np.sum(fake_counts)
gauss_counts,_  = np.histogram(gauss, bins = bins, density=True)
gauss_counts /= np.sum(gauss_counts)
eyesyn_counts,_  = np.histogram(eyesyn, bins = bins, density=True)
eyesyn_counts /= np.sum(eyesyn_counts)
stat_counts,_  = np.histogram(stat, bins = bins, density=True)
stat_counts /= np.sum(stat_counts)
vae_counts,_  = np.histogram(f_vae, bins = bins, density=True)
vae_counts /= np.sum(vae_counts)


js_divergence_vel_fake  = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(fake_counts) + epsilon)

js_divergence_vel_gauss = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(gauss_counts) + epsilon)

js_divergence_vel_real = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(real_counts_2) + epsilon)

js_divergence_vel_syneye = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(eyesyn_counts) + epsilon)

js_divergence_vel_stat = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(stat_counts) + epsilon)

js_divergence_vel_vae = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(vae_counts) + epsilon)


print('JS(real || GAN): ' + str(np.round(js_divergence_vel_fake,decimals = 3)))
print('JS(real || Gauss): ' + str(np.round(js_divergence_vel_gauss,decimals=3)))
print('JS(real || real): ' + str(np.round(js_divergence_vel_real,decimals = 3)))
print('JS(real || EyeSyn): ' + str(np.round(js_divergence_vel_syneye,decimals = 3)))
print('JS(real || stat): ' + str(np.round(js_divergence_vel_stat,decimals = 3)))
print('JS(real || VAE): ' + str(np.round(js_divergence_vel_vae,decimals = 3)))

JS(real || GAN): 0.22
JS(real || Gauss): 0.42
JS(real || real): 0.031
JS(real || EyeSyn): 0.811
JS(real || stat): 0.856
JS(real || VAE): 0.907


### amplitude
#### not normalized

In [43]:
real = real_amplitudes_sac
real_2 = np.array([np.mean(a) for a in real_amplitudes_sac_2])
fake = fake_amplitudes_sac
gauss = gauss_amplitudes_sac
f_vae = vae_amplitudes_sac

min_val = np.min(list(real) + list(fake) + list(gauss) + list(real_2) + list(f_vae))
max_val = np.max(list(real) + list(fake) + list(gauss) + list(real_2) + list(f_vae))
num_bins = 100
epsilon = 0.00001
bins = np.linspace(min_val,max_val,num_bins)

real_counts,_   = np.histogram(real, bins = bins, density=True)
real_counts_2,_   = np.histogram(real_2, bins = bins, density=True)
fake_counts,_   = np.histogram(fake, bins = bins, density=True)
gauss_counts,_  = np.histogram(gauss, bins = bins, density=True)
vae_counts,_  = np.histogram(f_vae, bins = bins, density=True)

js_divergence_vel_fake  = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(fake_counts) + epsilon)

js_divergence_vel_gauss = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(gauss_counts) + epsilon)

js_divergence_vel_real = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(real_counts_2) + epsilon)

js_divergence_vel_vae = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(vae_counts) + epsilon)

print('JS(real || GAN): ' + str(np.round(js_divergence_vel_fake,decimals = 2)))
print('JS(real || Gauss): ' + str(np.round(js_divergence_vel_gauss,decimals=2)))
print('JS(real || real): ' + str(np.round(js_divergence_vel_real,decimals = 2)))
print('JS(real || EyeSyn): ' + str(np.round(js_divergence_vel_syneye,decimals = 2)))
print('JS(real || VAE): ' + str(np.round(js_divergence_vel_vae,decimals = 2)))

JS(real || GAN): 1.12
JS(real || Gauss): 2.07
JS(real || real): 0.16
JS(real || EyeSyn): 0.81
JS(real || VAE): 4.78


#### normalized

In [44]:
real = real_amplitudes_sac
real_2 = np.array([np.mean(a) for a in real_amplitudes_sac_2])
fake = fake_amplitudes_sac
gauss = gauss_amplitudes_sac
f_vae = vae_amplitudes_sac

min_val = np.min(list(real) + list(fake) + list(gauss) + list(real_2) + list(f_vae))
max_val = np.max(list(real) + list(fake) + list(gauss) + list(real_2) + list(f_vae))
num_bins = 100
epsilon = 0.00001
bins = np.linspace(min_val,max_val,num_bins)

real_counts,_   = np.histogram(real, bins = bins, density=True)
real_counts /= np.sum(real_counts)
real_counts_2,_   = np.histogram(real_2, bins = bins, density=True)
real_counts_2 /= np.sum(real_counts_2)
fake_counts,_   = np.histogram(fake, bins = bins, density=True)
fake_counts /= np.sum(fake_counts)
gauss_counts,_  = np.histogram(gauss, bins = bins, density=True)
gauss_counts /= np.sum(gauss_counts)
eyesyn_counts,_  = np.histogram(eyesyn, bins = bins, density=True)
eyesyn_counts /= np.sum(eyesyn_counts)
vae_counts,_  = np.histogram(f_vae, bins = bins, density=True)
vae_counts /= np.sum(vae_counts)


js_divergence_vel_fake  = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(fake_counts) + epsilon)

js_divergence_vel_gauss = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(gauss_counts) + epsilon)

js_divergence_vel_real = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(real_counts_2) + epsilon)

js_divergence_vel_syneye = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(eyesyn_counts) + epsilon)

js_divergence_vel_vae = js_divergence(np.array(real_counts)  + epsilon,
                                      np.array(vae_counts) + epsilon)


print('JS(real || GAN): ' + str(np.round(js_divergence_vel_fake,decimals = 3)))
print('JS(real || Gauss): ' + str(np.round(js_divergence_vel_gauss,decimals=3)))
print('JS(real || real): ' + str(np.round(js_divergence_vel_real,decimals = 3)))
print('JS(real || EyeSyn): ' + str(np.round(js_divergence_vel_syneye,decimals = 3)))
print('JS(real || VAE): ' + str(np.round(js_divergence_vel_vae,decimals = 3)))

JS(real || GAN): 0.214
JS(real || Gauss): 0.395
JS(real || real): 0.03
JS(real || EyeSyn): 0.947
JS(real || VAE): 0.915
